## Семинар по библиотеке Numpy

Исследуем использование матриц для решения систем линейных уравнений. Например, система

$$
\begin{cases}
x_1+x_2+x_3 = 2 \\
2x_1+3x_2-x_3 = 5\\
x_1-x_2+x_3 = 0\\
\end{cases}
$$
может быть представлена в матричной форме
$$
\begin{pmatrix} 1 & 1 & 1 \\ 2 & 3 & -1 \\ 1 & -1 & 1\end{pmatrix} \times
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix} =
\begin{pmatrix} 2 \\ 5 \\ 0 \end{pmatrix}
$$

Сгенерируем матрицу для системы уравнений с 7-ю неизвестными:

In [69]:
import numpy as np
np.set_printoptions(precision=3)

A = np.random.uniform(1,5,size=(7,7)).astype(np.float64)
b = np.random.uniform(1,5,size=(7,)).astype(np.float64)

print(f"det(A)={np.linalg.det(A)}")

det(A)=112.04559610880213


Если определитель такой матрицы не равен 0, то уравнение будет иметь единственное решение. Иначе запустите ячейку ещё раз.

Из лекции мы помним, что можно найти решение с помощью обратной матрицы:

In [70]:
x = np.linalg.inv(A)@b
print(A)
print(x)
print(A@x)
print(b)

[[1.028 1.465 4.526 1.884 4.807 2.185 1.365]
 [4.054 2.178 3.173 2.572 2.363 3.325 2.876]
 [2.134 4.736 2.969 4.069 3.731 3.42  3.357]
 [1.199 4.528 3.126 3.485 4.824 3.528 1.378]
 [2.341 3.433 2.31  1.892 2.196 3.655 2.619]
 [4.402 1.322 2.885 1.23  2.275 4.278 4.6  ]
 [1.194 4.896 2.498 1.655 2.504 3.719 4.609]]
[ 3.754  4.268  2.666 -4.292  0.422 -4.372 -1.249]
[4.857 4.802 1.1   2.091 3.155 1.09  3.974]
[4.857 4.802 1.1   2.091 3.155 1.09  3.974]


Но как найти решение без использования алгоритма обращения матрицы? Для этого используется процесс **диагонализации**. Мы приводим систему уравнений к **эквивалентной системе**, путём вычитания строк, домноженных на какой-то коэффициент. Например, если во второй строке при $x_1$ стоит коэфициент 5, а в первой - 1, то вычтя из второй строки первую, домноженную на 5 - мы получим уравнение, в котором при $x_1$ стоит ноль. Далее, из второй строки мы вычитаем первые две, домноженые на определённые коэффициенты, приводя коэффициенты при $x_1$ и $x_2$ к нулю. И так далее. В последнем уравнении будет ненулевой коэффициент только при $x_n$.

Мы также сразу будем делить $i$-ю строчку матрицы на $A_{ii}$, чтобы получить на диагонали единички:

In [71]:
def diagonalize(A,b):
    n = A.shape[0]
    for i in range(n):
        for j in range(i):
            coef = A[i,j]/A[j,j]
            A[i] = A[i]-coef*A[j]
            b[i] = b[i]-coef*b[j]
        b[i] = b[i]/A[i,i]
        A[i] = A[i]/A[i,i]

diagonalize(A,b)
print(A)
print(b)

[[ 1.     1.425  4.405  1.834  4.678  2.126  1.329]
 [-0.     1.     4.079  1.351  4.612  1.471  0.697]
 [-0.    -0.     1.     0.16   1.054  0.27   0.049]
 [-0.    -0.    -0.     1.    -1.795 -1.552  4.443]
 [-0.    -0.    -0.    -0.     1.     0.418 -2.068]
 [ 0.     0.     0.     0.     0.     1.    11.207]
 [ 0.     0.     0.     0.     0.     0.     1.   ]]
[  4.726   3.989   1.18   -3.816   1.178 -18.371  -1.249]


Этот алгоритм не всегда завершается успехом, например, если в процессе диагонализации при каком-то их $x_i$ будет получен нулевой коэффициент - может возникнуть ошибка деления на 0. В этом случае используют специальные приёмы перестановки строк - но мы не будем вдаваться в эти подробности. Если у вас возникнет ошибка - просто пересоздайте случайную матрицу ещё раз.

> Обратите внимание, что сложение и умножение строк матрицы осуществляется одним выражением, например `A[i] = A[i]-coef*A[j]`. Здесь мы модифицируем целиком $i$-ю строку матрицы.

Мы уже получили значение $x_n$ - поскольку в матрице в последней строке коэффициент при $x_n$ равен 1, то значение $x_n$ - это $b_n$. Чтобы получить значения всех $x_i$, необходимо проделать аналогичный процесс диагонализации для верхней части матрицы:

In [72]:
def solve(A,b):
    n = A.shape[0]
    for i in range(n-2,-1,-1):
        for j in range(i+1,n):
            coef = A[i,j]
            A[i]=A[i]-A[j]*coef
            b[i]=b[i]-b[j]*coef

solve(A,b)
print(A)
print(b)

[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [-0. -0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]
[ 3.754  4.268  2.666 -4.292  0.422 -4.372 -1.249]
